In [1]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

import json

# Rewriting the Draft Algorithm with Linear Alegebra Operations

In [2]:
N_DRAFTERS = 4

M19_DECK_ARCHYTYPES = ["WR", "UB", "GR", "WB", "UG", "WU", "BR", "WG", "UR", "BG"]
N_ARCHETYPES = len(M19_DECK_ARCHYTYPES)

In [3]:
CARD_VALUES_DICT = json.load(open('data/m19-custom-card-values-tuples-reduced.json'))['values']
CARD_NAMES = list(CARD_VALUES_DICT)
N_CARDS = len(CARD_NAMES)

`player_preferences` is an `n_draters * n_arhetypes` array that tracks the player's preference for each archetype.  It is updated each time a player makes a draft pick.

In [4]:
player_prefercences = np.ones(shape=(N_DRAFTERS, N_ARCHETYPES))

`archetype_weights` is a `n_cards * n_archetypes` array containing the rating data for each card in each archetype.  This is what we would want a ML model to learn from actual draft data.

In [5]:
archetype_weights_df = pd.DataFrame(CARD_VALUES_DICT).T
archetype_weights_df.columns = M19_DECK_ARCHYTYPES

archetype_weights = archetype_weights_df.values

assert(archetype_weights.shape == (N_CARDS, N_ARCHETYPES))

`cards_in_pack` is an `n_drafters * n_cards` array that indicates which cards are in the players current pack.  Repeated cards are represented as counts.

In [6]:
from draftbot import Set

In [7]:
m19_reduced_data = json.load(open('data/m19-subset-reduced.json'))
m19_reduced_card_values = json.load(open('data/m19-custom-card-values-reduced.json'))
m19 = Set(cards=m19_reduced_data, card_values=m19_reduced_card_values)

TypeError: __init__() got an unexpected keyword argument 'card_values'

In [ ]:
packs = [m19.random_pack_dict() for _ in range(N_DRAFTERS)]

cards_in_pack_df = pd.DataFrame(np.zeros(shape=(N_DRAFTERS, N_CARDS), dtype=int), 
                                columns=CARD_NAMES)

# This is probably inefficient...
for idx, pack in enumerate(packs):
    for card in pack:
        name = card['name']
        cards_in_pack_df.loc[cards_in_pack_df.index[idx], name] += 1

cards_in_pack = cards_in_pack_df.values

`card_is_in_pack` is an `n_drafters * n_cards` array that simply indicates if a card is in the pack.

In [ ]:
card_is_in_pack = np.sign(cards_in_pack)

`pack_archetype_weights` is a `n_drafters * n_cards * n_archeypes` array that has the same zero/non-zero structure as `cards_in_pack_exploded`.  A non-zero entry in the `(d, c, a)` is the weight for card `c` in arhetype `a`.  A zero entry in the `(d, c, a)` position indicates that the card `c` is not available for drafter `d` in the current pack. 

In [ ]:
pack_archetype_weights = (
    card_is_in_pack.reshape((N_DRAFTERS, N_CARDS, 1)) * 
    archetype_weights.reshape((1, N_CARDS, N_ARCHETYPES)))

assert(pack_archetype_weights.shape == (N_DRAFTERS, N_CARDS, N_ARCHETYPES))

`preferences` is an `n_drafters * n_cards` array containing the total preference for each drafters cards in their current pack.  It is a tensor product between the `pack_archetype_weights` and `player_preferences` array, with the shared `n_arhetype` dimensions contracted together.

In [ ]:
preferences = np.einsum('dca,da->dc', pack_archetype_weights, player_prefercences)

In [ ]:
preferences_df = pd.DataFrame(preferences, columns=CARD_NAMES)
preferences_df

Softmaxing the `preferences` array results in the probability of the drafters picking each card in their pack.

In [ ]:
def softmax(x):
    exps = np.exp(x)
    row_sums = np.sum(exps, axis=1)
    probs = exps / row_sums.reshape(-1, 1)
    return probs

pick_probs = softmax(preferences)

In [ ]:
pick_probs_df = pd.DataFrame(np.round(pick_probs, 2), columns=CARD_NAMES)
pick_probs_df

Now we actually make a pick by choosing a card according to the probability distribution in each row.

In [ ]:
pick = np.zeros((N_DRAFTERS, N_CARDS), dtype=int)

for ridx, row in enumerate(pick_probs):
    pick_idx = np.random.choice(N_CARDS, p=row)
    pick[ridx, pick_idx] = 1

In [ ]:
pick_df = pd.DataFrame(pick, columns=CARD_NAMES)
pick_df

### Update Rules

Now we update the two data structures:
    
  - The `cards_in_pack` array is updated by removing one copy of the picked card from each pack.  Then each pack goes to the next player in line, which we accomplish by rotating the rows of the `cards_in_pack` array.
  - The `player_preferences` array is updated by adding the archetype weights of the chosen card to the player's row.

In [ ]:
cards_in_pack_new = card_is_in_pack - pick

In [ ]:
cards_in_pack_new_df = pd.DataFrame(cards_in_pack_new, columns=CARD_NAMES)
cards_in_pack_new_df

In [ ]:
player_preferences_new = player_prefercences + np.einsum(
    'ca,pc->pa', archetype_weights, pick)

In [ ]:
player_preferences_new_df = pd.DataFrame(player_preferences_new, columns=M19_DECK_ARCHYTYPES)
player_preferences_new_df

In [ ]:
x = np.array([[1, 1], [2, 2], [3, 3], [4, 4]])

In [ ]:
def rotate_array(x, forwards=True):
    newx = np.zeros(x.shape)
    if forwards:
        newx[0, :] = x[-1, :]
        newx[1:, :] = x[:-1, :]
    else:
        newx[-1, :] = x[0, :]
        newx[:-1, :] = x[1:, :]
    return newx

In [ ]:
rotate_array(x, False)